In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv("bug_data_exported.csv")
df.head()

,id,description,severity
0,91,document properties cannot be listed,major
1,103,layout bug: table cell overflows containing cell,normal
2,104,table saved as text: missing inter-column space,minor
3,105,nested <TABLE>s: bgcolor of inner table not re...,normal
4,133,Navigator draws entities like &lt;&amp;&gt; as...,minor


In [3]:
df = df.dropna()


In [4]:
df.severity.value_counts()

normal      272520
major        35898
critical     31953
minor        19451
blocker       3694
Name: severity, dtype: int64

In [5]:
# df['severity_num'] = df.severity.map({'minor':0,'normal':1,'major':2,'critical':4,'blocker':5})
df['severity_num'] = df.severity.map({'minor':0,'normal':1,'major':2,'critical':3})


In [6]:
df_normal = df[df.severity_num==1]
df_major = df[df.severity_num==2]
df_critical = df[df.severity_num==3]
df_minor = df[df.severity_num==0]
# df_blocker = df[df.severity_num==5]

In [7]:
from sklearn.utils import resample
def resample_df(df):
    return resample(df,replace=True,n_samples=5000,random_state=123)

df_normal = resample_df(df_normal)
df_major = resample_df(df_major)
df_critical = resample_df(df_critical)
df_minor = resample_df(df_minor)
# df_blocker = resample_df(df_blocker)

# df = pd.concat([df_normal,df_major,df_critical,df_minor,df_blocker])
df = pd.concat([df_normal,df_major,df_critical,df_minor])

df.severity.value_counts()


normal      5000
major       5000
critical    5000
minor       5000
Name: severity, dtype: int64

In [8]:
#define x as bug description and y as the severity
x = df.description
y = df.severity_num
print(x.shape)
print(y.shape)

(20000,)
(20000,)


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33)

y_test.value_counts()


2.0    1673
1.0    1660
0.0    1656
3.0    1611
Name: severity_num, dtype: int64

In [10]:
#instantiate the vectorizer
vect = CountVectorizer()
vect.fit(x_train)
x_train_dtm = vect.transform(x_train)
tfdf_transformer = TfidfTransformer()
x_train_tfdf = tfdf_transformer.fit_transform(x_train_dtm)

In [11]:
svm = SVC(kernel='rbf',C=6,gamma=0.1)
svm.fit(x_train_tfdf,y_train)

SVC(C=6, gamma=0.1)

In [12]:
x_test_dtm = vect.transform(x_test)
x_test_tfdf = tfdf_transformer.transform(x_test_dtm)

In [13]:
y_pred = svm.predict(x_test_tfdf)

In [14]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.50      0.56      0.53      1656
         1.0       0.43      0.37      0.40      1660
         2.0       0.43      0.45      0.44      1673
         3.0       0.74      0.72      0.73      1611

    accuracy                           0.52      6600
   macro avg       0.52      0.52      0.52      6600
weighted avg       0.52      0.52      0.52      6600

